In [ ]:
import obspy
import os
import glob
from IPython.display import clear_output

In [ ]:
root_dir = "/media/elemento/Element/NTU_RW/NEFS"
os.listdir(root_dir)

In [ ]:
def do_resample(data_dir, sample_rate):
    """
    resample data
    """
    if not os.path.exists(data_dir):
        print("Missing File: %s" % data_dir)

    for ev in os.listdir(data_dir):
        print(ev)
        inpath = os.path.join(data_dir, ev)
        for sac in glob.glob(f"{inpath}/*.SAC"):
            try:
                st = obspy.read(sac)
                print(st)

                # Resample
                st.resample(sample_rate, no_filter="False")
                st[0].write(sac, format='SAC')
                
                clear_output(wait=False)

            except:
                continue

In [ ]:
def do_filter(data_dir, freq_low, freq_high, phase):
    """
    filter data
    """
    if not os.path.exists(data_dir):
        print("Missing File: %s" % data_dir)

    for ev in os.listdir(data_dir):
        print(ev)
        inpath = os.path.join(data_dir, ev)
        # Remove the existed filtered files *.bp
        for bp in glob.glob(f"{inpath}/*.bp*"):
            os.remove(bp)
        
        for sac in glob.glob(f"{inpath}/*.SAC"):
            try:
                st = obspy.read(sac)
                print(st)

                # rmean, rtrend & taper
                st.detrend("demean")
                st.detrend("linear")
                st.taper(max_percentage=0.05, type='hann')

                # Bandpass
                st.filter('bandpass', freqmin=freq_low, freqmax=freq_high, corners=2, zerophase=phase)

                st[0].write(sac+".bp", format='SAC')
                
                clear_output(wait=False)
                
            except:
                continue

In [ ]:
# Filter data
providers = ["IRIS", "GFZ"]
years = [2000, 2001, 2002, 2003]

for year1 in years:
    year2 = year1 + 1
    for provider in providers:
        print(f"For the range: {year1}-{year2}, and provider: {provider}")
        
        data_sac = f'{root_dir}/Download/{provider}/SAC/{year1}'
        
        # Perform Resampling, Unified Resampling Rate (20Hz)
        do_resample(data_sac, 20)
        
        # Applying Bandpass Filter, 0.02 Hz - 2 Hz
        do_filter(data_sac, 0.02, 2.0, "TRUE")
        
        clear_output(wait=False)